In [3]:
# from pyswarm import pso
import pandas as pd
import warnings 
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from sklearn import preprocessing
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
import random
import numpy as np
import matplotlib.pyplot as plt
# from sko.PSO import PSO
from xgboost import XGBClassifier

In [5]:
data_trains = pd.read_excel('Molecular_Descriptor.xlsx',sheet_name='training')
data_trains_feature = pd.DataFrame(data_trains,columns=['MDEC-23','MLogP','LipoaffinityIndex','minsOH','nC','minsssN','CrippenLogP',
                                               'maxHsOH','C1SP2','minHsOH','maxsOH','nT6Ring','n6Ring','BCUTp-1h',
                                               'C2SP2','hmin','AMR','SwHBa','maxsssN','MDEC-22'])
data_trains_feature.to_excel('data_select_pso.xlsx')

In [ ]:
data_f = pd.read_excel('Molecular_Descriptor_select_feature.xlsx',index_col=0,engine='openpyxl')
data_label_cls = pd.read_excel('ADMET_final_label.xlsx',index_col=0,engine='openpyxl')
data_label_reg = pd.read_excel('ER_activity.xlsx',index_col=0,engine='openpyxl')
#根据(1973,20)数据训练出一个回归器，再使用粒子群算法优化该回归器
rfr = RandomForestRegressor(n_estimators=1000,max_depth=100)
rfr.fit(data_f,data_label_reg['pIC50'])
print('随机森林回归训练完成')
xgb = XGBClassifier(n_estimators=100,gamma=0.1,max_depth=4,min_child_weight=3)
xgb.fit(data_f,data_label_cls['Fina_label'])
print('xgboost分类训练完成')
#根据（1973，20）数据训练出一个二分类器，作为约束条件



def fitness(ind_var):
    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20 = ind_var
    # x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20 = ind_var[0],ind_var[1],ind_var[2],ind_var[3],
    # ind_var[4],ind_var[5],ind_var[6],ind_var[7],ind_var[8],ind_var[9],ind_var[10],ind_var[11],ind_var[12],ind_var[13],
    # ind_var[14],ind_var[15],ind_var[16],ind_var[17],ind_var[18],ind_var[19]
    data_pre = np.array([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20]).reshape(1,-1)
    y = rfr.predict(data_pre)
    return -float(y)#添加负号最大化目标函数
# list(data_f.min())
#     up = list(data_f.max())
cond = (
    lambda x:xgb.predict(x)
)
print('开始执行PSO算法')
pso = PSO(func=fitness,dim=20,pop=60, max_iter=1, lb=list(data_f.min()), ub=list(data_f.max()), w=0.8, c1=0.5, c2=0.5,constraint_eq=cond)
pso.record_mode=True
pso.run()
print('best_x is ', pso.gbest_x, 'best_y is', pso.gbest_y)
print(pso.record_values)